In [3]:
import numpy as np
import pandas as pd
from scipy import stats
import sys
import argparse
import h5py
from scipy.stats import t as student_t
from statsmodels.stats import multitest as mt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [10]:
# IDP_df = pd.read_pickle('male_IDP.pkl')
vars_df = pd.read_pickle('female_vars.pkl')
# IDP_names = np.loadtxt("IDP_names.txt", dtype=str, delimiter='\n')
# IDP_categories = np.loadtxt("IDP_categories.txt", dtype=str, delimiter='\n')
with open('../Analsysis Female New/varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
# vars_categories = np.loadtxt("../Analsysis Female New/vars_categories.txt", dtype=str, delimiter='\n')
# vars_categories = np.loadtxt("../Analsysis Female New/vars_categories.txt", dtype=str)

with open('../Analsysis Female New/vars_categories.txt') as f:
    vars_categories = f.readlines()
    vars_categories = [l.strip('\n\r') for l in vars_categories]
    vars_categories = np.array(vars_categories)

In [13]:
vars_df.head()

,modality,bonf,fdr,dataframe
0,T1L_large_HGL,5.441192,3.425256,idx ...
1,T1L_large_SFCN,5.441192,3.113612,idx ...
2,T1L_large_SFCN_50b,5.441192,3.159399,idx ...
3,T1L_large_SFCN_50b10s,5.441192,3.236979,idx ...


In [15]:
df = vars_df.iloc[0].dataframe

In [19]:
df[df.log_p_values > 2]

,idx,names,Categories,pearson_r,t_test_statistic,p_values_corrected,p_values,abs_pearson_r,log_p_values,log_p_values_corrected
132,160,Father still alive (0.0),Lifestyle,-0.040368,2.832381,0.261474,0.004639,0.040368,2.333568,0.582572
149,182,Age first had sexual intercourse (0.0),Lifestyle,-0.050758,3.475740,0.059654,0.000514,0.050758,3.288972,1.224357
151,184,Age first had sexual intercourse (2.0),Lifestyle,-0.046285,3.072440,0.172505,0.002136,0.046285,2.670365,0.763199
152,186,Lifetime number of sexual partners (0.0),Lifestyle,0.054113,3.594708,0.045337,0.000328,0.054113,3.483710,1.343547
154,188,Lifetime number of sexual partners (2.0),Lifestyle,0.046755,2.990476,0.201530,0.002802,0.046755,2.552522,0.695661
...,...,...,...,...,...,...,...,...,...,...
13233,16845,Operative procedures - OPCS4 (T313 - T31.3 Exc...,Medical History,0.040901,2.900029,0.234121,0.003747,0.040901,2.426261,0.630560
13311,16923,Operative procedures - OPCS4 (U051 - U05.1 Com...,Medical History,0.041120,2.915599,0.227962,0.003566,0.041120,2.447846,0.642137
13337,16949,Operative procedures - OPCS4 (U211 - U21.1 Mag...,Medical History,0.047614,3.377058,0.078115,0.000738,0.047614,3.131810,1.107264
13356,16968,Operative procedures - OPCS4 (V256 - V25.6 Pri...,Medical History,0.039548,2.803999,0.273305,0.005067,0.039548,2.295275,0.563353


In [21]:
vars_df.iloc[0].dataframe.head()

,idx,names,Categories,pearson_r,t_test_statistic,p_values_corrected,p_values,abs_pearson_r,log_p_values,log_p_values_corrected
0,0,Ethnic background (0.0),Ethnic Background,-0.017822,1.261440,0.879211,0.207209,0.017822,0.683591,5.590694e-02
1,1,Ethnic background (1.0),Ethnic Background,-0.013354,0.386143,1.000000,0.699489,0.013354,0.155219,4.821637e-17
2,2,Ethnic background (2.0),Ethnic Background,-0.023751,0.896215,0.970520,0.370290,0.023751,0.431459,1.299556e-02
3,3,Genotype measurement batch (0.0),Genetic Markers,-0.004550,0.317799,1.000000,0.750651,0.004550,0.124562,4.821637e-17
4,4,Heterozygosity (0.0),Genetic Markers,-0.008424,0.588403,1.000000,0.556289,0.008424,0.254699,4.821637e-17


In [22]:
vars_over_bThr_list = []

for idx in range(len(vars_df)):
    df = vars_df.iloc[idx].dataframe
    bThr = vars_df.iloc[idx].bonf
    vars_over_bThr = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False).names.to_list()
    for var in vars_over_bThr:
        vars_over_bThr_list.append(var)

In [23]:
print("Number of unique vars across all modalities: ", len(set(vars_over_bThr_list)))

Number of unique vars across all modalities:  105


In [24]:
unique_vars_over_bTHr = list(set(vars_over_bThr_list))

In [25]:
unique_vars_over_bThr_categories = []

for var in unique_vars_over_bTHr:
    idx = list(varsHeader).index(var)
    category = vars_categories[idx]
    unique_vars_over_bThr_categories.append(category)

In [26]:
sorted(unique_vars_over_bTHr)

['Alcohol (2.0)',
 'Alcohol intake frequency. (0.0)',
 'Alcohol intake frequency. (2.0)',
 'Amount of alcohol drunk on a typical drinking day (0.0)',
 'Arm BMD (bone mineral density) (left) (2.0)',
 'Arm BMD (bone mineral density) (right) (2.0)',
 'Arm fat mass (right) (2.0)',
 'Arm fat percentage (left) (2.0)',
 'Arm fat percentage (right) (2.0)',
 'Arms BMC (bone mineral content) (2.0)',
 'Arms BMD (bone mineral density) (2.0)',
 'Arms total mass (2.0)',
 'Average weekly red wine intake (0.0)',
 'Body fat percentage (0.0)',
 'Body fat percentage (2.0)',
 'Body mass index (BMI) (2.0)',
 'Diabetes diagnosed by doctor (0.0)',
 'Diagnoses - ICD10 (C770 - C77.0 Lymph nodes of head, face and neck)',
 'Diagnoses - ICD10 (E109 - E10.9 Without complications)',
 'Diagnoses - ICD10 (H654 - H65.4 Other chronic nonsuppurative otitis media)',
 'Diagnoses - ICD10 (I10 - I10 Essential (primary) hypertension)',
 'Diagnoses - main ICD10 (G409 - G40.9 Epilepsy, unspecified)',
 'Diagnoses - secondary IC

In [27]:
modalities = []
pearson_rs_list = []
for var in unique_vars_over_bTHr:
    modalities_with_var = []
    pearsonRs_with_var = []
    for idx in range(len(vars_df)):
        df = vars_df.iloc[idx].dataframe
        modality = vars_df.iloc[idx].modality
        bThr = vars_df.iloc[idx].bonf
        df = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False)
        vars_over_bThr = df.names.to_list()
        pearson_r_over_bThr = df.pearson_r.to_list()
        if var in vars_over_bThr:
            modalities_with_var.append(modality)
            pearsonRs_with_var.append(pearson_r_over_bThr[vars_over_bThr.index(var)])
    modalities.append(modalities_with_var)
    pearson_rs_list.append(pearsonRs_with_var)

In [28]:
len(modalities)

105

In [29]:
df_vars_over_bThr = pd.DataFrame.from_dict({
    'names': unique_vars_over_bTHr,
    'Categories': unique_vars_over_bThr_categories,
    'modalities': modalities,
    'pearson_r': pearson_rs_list
})

In [30]:
df_vars_over_bThr.head()

,names,Categories,modalities,pearson_r
0,Treatment/medication code (1140883066 - insuli...,Medical History,[T1L_large_SFCN_50b],[0.06565669144462362]
1,Trunk BMC (bone mineral content) (2.0),Skeletal Measurements,"[T1L_large_HGL, T1L_large_SFCN, T1L_large_SFCN...","[-0.09970406449475275, -0.15335192466080955, -..."
2,Whole body fat mass (2.0),Physical Measurements,"[T1L_large_HGL, T1L_large_SFCN, T1L_large_SFCN...","[-0.06905512362720176, -0.08138332229866313, -..."
3,Speed of sound through heel (left) (0.0),Skeletal Measurements,"[T1L_large_SFCN, T1L_large_SFCN_50b, T1L_large...","[-0.14211822035477967, -0.14252042869653445, -..."
4,Femur lower neck BMD (bone mineral density) (r...,Skeletal Measurements,"[T1L_large_SFCN, T1L_large_SFCN_50b, T1L_large...","[-0.22862967365857334, -0.22330846076241476, -..."


In [31]:
df_vars_over_bThr[df_vars_over_bThr.names=='Arm fat mass (right) (0.0)']

,names,Categories,modalities,pearson_r


In [32]:
set(df_vars_over_bThr.Categories.to_list())

{'Alcohol',
 'Blood Assays',
 'Cardiac & Circulartory Measurements',
 'Diet',
 'Lifestyle',
 'Medical History',
 'Physical Measurements',
 'Skeletal Measurements',
 'Tobacco'}

In [33]:
for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        print('Var: {}'.format(name))
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        print('   Modalities:')
        for i, mod in enumerate(mods[0]):
            print('   -> {} ({:.3f})'.format(mod, prs[0][i]))
        
    print('\n')

Category: Alcohol
------------------------------------------------
Var: Alcohol intake frequency. (0.0)
   Modalities:
   -> T1L_large_SFCN (0.093)
   -> T1L_large_SFCN_50b (0.097)
   -> T1L_large_SFCN_50b10s (0.086)
Var: Alcohol intake frequency. (2.0)
   Modalities:
   -> T1L_large_SFCN (0.109)
   -> T1L_large_SFCN_50b (0.105)
   -> T1L_large_SFCN_50b10s (0.094)
Var: Amount of alcohol drunk on a typical drinking day (0.0)
   Modalities:
   -> T1L_large_SFCN_50b (0.081)
Var: Average weekly red wine intake (0.0)
   Modalities:
   -> T1L_large_SFCN_50b (0.079)
Var: Frequency of consuming six or more units of alcohol (0.0)
   Modalities:
   -> T1L_large_SFCN (0.081)
   -> T1L_large_SFCN_50b (0.084)
Var: Frequency of drinking alcohol (0.0)
   Modalities:
   -> T1L_large_SFCN (0.131)
   -> T1L_large_SFCN_50b (0.128)
   -> T1L_large_SFCN_50b10s (0.108)


Category: Blood Assays
------------------------------------------------
Var: Mean corpuscular volume (0.0)
   Modalities:
   -> T1L_large_

In [34]:
sorted(prs[0], reverse=True)

[0.07000262529127813, 0.06676064027415653]

In [35]:
for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        print('Var: {}'.format(name))
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        print('   Modalities:')
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        
        for i, mod in enumerate(mods):
            print('   -> {} ({:.3f})'.format(mod, prs[i]))
        
    print('\n')

Category: Alcohol
------------------------------------------------
Var: Alcohol intake frequency. (0.0)
   Modalities:
   -> T1L_large_SFCN_50b (0.097)
   -> T1L_large_SFCN (0.093)
   -> T1L_large_SFCN_50b10s (0.086)
Var: Alcohol intake frequency. (2.0)
   Modalities:
   -> T1L_large_SFCN (0.109)
   -> T1L_large_SFCN_50b (0.105)
   -> T1L_large_SFCN_50b10s (0.094)
Var: Amount of alcohol drunk on a typical drinking day (0.0)
   Modalities:
   -> T1L_large_SFCN_50b (0.081)
Var: Average weekly red wine intake (0.0)
   Modalities:
   -> T1L_large_SFCN_50b (0.079)
Var: Frequency of consuming six or more units of alcohol (0.0)
   Modalities:
   -> T1L_large_SFCN_50b (0.084)
   -> T1L_large_SFCN (0.081)
Var: Frequency of drinking alcohol (0.0)
   Modalities:
   -> T1L_large_SFCN (0.131)
   -> T1L_large_SFCN_50b (0.128)
   -> T1L_large_SFCN_50b10s (0.108)


Category: Blood Assays
------------------------------------------------
Var: Mean corpuscular volume (0.0)
   Modalities:
   -> T1L_large_

In [36]:
variance_threshold = 0.1

for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        
        if prs[0] >= variance_threshold or abs(prs[-1])>= variance_threshold:
            print('Var: {}'.format(name))
            print('   Modalities:')
            for i, mod in enumerate(mods):
                if prs[i] >= variance_threshold or abs(prs[i])>= variance_threshold:
                    print('   -> {} ({:.3f})'.format(mod, prs[i]))
        
    print('\n')

Category: Alcohol
------------------------------------------------
Var: Alcohol intake frequency. (2.0)
   Modalities:
   -> T1L_large_SFCN (0.109)
   -> T1L_large_SFCN_50b (0.105)
Var: Frequency of drinking alcohol (0.0)
   Modalities:
   -> T1L_large_SFCN (0.131)
   -> T1L_large_SFCN_50b (0.128)
   -> T1L_large_SFCN_50b10s (0.108)


Category: Blood Assays
------------------------------------------------


Category: Cardiac & Circulartory Measurements
------------------------------------------------


Category: Diet
------------------------------------------------
Var: Alcohol (2.0)
   Modalities:
   -> T1L_large_SFCN_50b10s (0.122)
   -> T1L_large_SFCN_50b (0.117)


Category: Lifestyle
------------------------------------------------


Category: Medical History
------------------------------------------------


Category: Physical Measurements
------------------------------------------------


Category: Skeletal Measurements
------------------------------------------------
Var: Arm BM

In [37]:
mods

('T1L_large_SFCN_50b', 'T1L_large_SFCN')

In [38]:
counter = 0
for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()[0]
        if 'T1_nonlinear' in mods or 'T1_linear' in mods or 'T2_nonlinear' in mods:
            pass
        else:
            print('Var: {}'.format(name))
            counter += 1
    print('\n')
    
print("There are {} vars independent from the conventional modalities".format(counter))

Category: Alcohol
------------------------------------------------
Var: Alcohol intake frequency. (0.0)
Var: Alcohol intake frequency. (2.0)
Var: Amount of alcohol drunk on a typical drinking day (0.0)
Var: Average weekly red wine intake (0.0)
Var: Frequency of consuming six or more units of alcohol (0.0)
Var: Frequency of drinking alcohol (0.0)


Category: Blood Assays
------------------------------------------------
Var: Mean corpuscular volume (0.0)


Category: Cardiac & Circulartory Measurements
------------------------------------------------
Var: Diastolic blood pressure, automated reading (0.0)
Var: Diastolic blood pressure, automated reading (0.1)


Category: Diet
------------------------------------------------
Var: Alcohol (2.0)


Category: Lifestyle
------------------------------------------------
Var: Lifetime number of sexual partners (0.0)
Var: Lifetime number of sexual partners (2.0)


Category: Medical History
------------------------------------------------
Var: Diabet

In [39]:
counter = 0
for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()[0]
        if 'T1_nonlinear' in mods or 'T1_linear' in mods or 'T2_nonlinear' in mods:
            print('Var: {}'.format(name))
            counter += 1
        else:
            pass
    print('\n')
    
print("There are {} vars in the conventional modalities".format(counter))

Category: Alcohol
------------------------------------------------


Category: Blood Assays
------------------------------------------------


Category: Cardiac & Circulartory Measurements
------------------------------------------------


Category: Diet
------------------------------------------------


Category: Lifestyle
------------------------------------------------


Category: Medical History
------------------------------------------------


Category: Physical Measurements
------------------------------------------------


Category: Skeletal Measurements
------------------------------------------------


Category: Tobacco
------------------------------------------------


There are 0 vars in the conventional modalities
